In [5]:
import pandas as pd
import numpy as np

In [47]:
%%bash
source ~/.bashrc
conda activate ipyrad
mkdir -p /storage/group/ibb3/default/darpa/linkage_scaff
cd /storage/group/ibb3/default/darpa/linkage_scaff
bwa mem \
    /storage/group/ibb3/default/genome_annotation/Apalm_dovetail_assemblies/Apalm_assembly_v2.0_180910.softmasked.fa \
    ../../convert_snpchip_coords/probes.fasta \
    > ./probes_aln_apal.sam

ERROR: This cross-compiler package contains no program /storage/group/ibb3/default/.conda/envs/ipyrad/bin/x86_64-conda_cos6-linux-gnu-cc
ERROR: activate-gcc_linux-64.sh failed, see above for details


[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 53589 sequences (3803707 bp)...
[M::mem_process_seqs] Processed 53589 reads in 3.243 CPU sec, 3.253 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem /storage/group/ibb3/default/genome_annotation/Apalm_dovetail_assemblies/Apalm_assembly_v2.0_180910.softmasked.fa ../../convert_snpchip_coords/probes.fasta
[main] Real time: 4.213 sec; CPU: 3.573 sec


In [48]:
%%bash
cd /storage/group/ibb3/default/darpa/linkage_scaff
cut -f1,3,4,5,6,10,12 probes_aln_apal.sam \
    > unique_mappings_apal_probes.tab

In [7]:
probes = pd.read_csv("/storage/group/ibb3/default/darpa/linkage_scaff/unique_mappings_apal_probes.tab", comment = "@", sep = "\t", names = ["probe", "contig", "start", "mapq", "cigar", "seq", "mismatch"])

In [9]:
probes = probes[~probes["cigar"].str.contains("H")].reset_index(drop=True)

In [10]:
probes["new_snp_pos"] = np.nan

In [11]:
for i in range(len(probes)):
    probes["new_snp_pos"][i] = probes["start"][i] + probes["seq"][i].find("N")

/tmp/ipykernel_2024491/2714359173.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  probes["new_snp_pos"][i] = probes["start"][i] + probes["seq"][i].find("N")


In [12]:
probes = probes[~probes["cigar"].str.contains("D")][~probes["cigar"].str.contains("I")].reset_index(drop=True)

/tmp/ipykernel_2024491/1942152665.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  probes = probes[~probes["cigar"].str.contains("D")][~probes["cigar"].str.contains("I")].reset_index(drop=True)


In [13]:
probes["new_snp_pos"] = probes["new_snp_pos"].astype(int)

In [15]:
recommended = pd.read_csv("/storage/group/ibb3/default/acropora_linkage_maps/Apalm_linkage_Map/plate1-2/recommended.txt", header = None)

In [17]:
probes = probes[probes["probe"].isin(recommended[0].tolist())].reset_index(drop=True)

In [18]:
file = open('/storage/group/ibb3/default/darpa/linkage_scaff/probes_in_apal.txt','w')
for item in probes["probe"].tolist():
    file.write(item+"\n")
file.close()

In [19]:
%%bash
source ~/.bashrc
conda activate ipyrad
cd /storage/group/ibb3/default/darpa/linkage_scaff/
vcftools --vcf /storage/group/ibb3/default/acropora_linkage_maps/Apalm_linkage_Map/plate1-2/Lep-MAP3/Apalm_link_rename.vcf \
    --snps probes_in_apal.txt \
    --recode --out ./conserved_probes_kate

ERROR: This cross-compiler package contains no program /storage/group/ibb3/default/.conda/envs/ipyrad/bin/x86_64-conda_cos6-linux-gnu-cc
ERROR: activate-gcc_linux-64.sh failed, see above for details



VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf /storage/group/ibb3/default/acropora_linkage_maps/Apalm_linkage_Map/plate1-2/Lep-MAP3/Apalm_link_rename.vcf
	--out ./conserved_probes_kate
	--recode
	--snps probes_in_apal.txt

After filtering, kept 107 out of 107 Individuals
Outputting VCF file...
After filtering, kept 23218 out of a possible 26075 Sites
Run Time = 7.00 seconds


In [21]:
reformat = probes[["contig", "start", "probe"]]
reformat.columns = ["contig", "start", "ID"]

In [22]:
%%bash
cd /storage/group/ibb3/default/darpa/linkage_scaff/
grep -v "^##" ./conserved_probes_kate.recode.vcf | cut -f1-3 \
    > ./original_coords_kate.tab

In [23]:
tmp = pd.read_csv("/storage/group/ibb3/default/darpa/linkage_scaff/original_coords_kate.tab", sep = "\t")

In [24]:
tmp = tmp.merge(reformat, how = "left", on = "ID")

In [25]:
reformatted_positions = tmp[["contig", "start", "ID"]]
reformatted_positions.columns = ["#CHROM", "POS", "ID"]

In [26]:
reformatted_positions.set_index("#CHROM").to_csv("/storage/group/ibb3/default/darpa/linkage_scaff/remapped_coords_kate.tab", sep = "\t")

In [27]:
%%bash
cd /storage/group/ibb3/default/darpa/linkage_scaff/
grep -e "^##fileformat" -e "^##FORMAT" -e "^##INFO" conserved_probes_kate.recode.vcf \
    > acer_snpchip_remapped_kate.vcf

In [28]:
%%bash
cd /storage/group/ibb3/default/darpa/linkage_scaff/
paste /storage/group/ibb3/default/convert_snpchip_coords/remapped_coords_kate.tab \
    <( grep -v "^##" conserved_probes_kate.recode.vcf | cut -f4- ) \
    >> acer_snpchip_remapped_kate.vcf

In [36]:
%%bash
cd /storage/group/ibb3/default/darpa/linkage_scaff/
grep "^#" acer_snpchip_remapped_kate.vcf > apal_snpchip_remapped_kate_sorted.vcf
grep -v "^#" acer_snpchip_remapped_kate.vcf | sort -k1,1V -k2,2g >> apal_snpchip_remapped_kate_sorted.vcf

In [37]:
%%bash
source ~/.bashrc
conda activate ipyrad
cd /storage/group/ibb3/default/darpa/linkage_scaff/

for i in apal_snpchip_remapped_kate_sorted.vcf; do
    vcftools --vcf ${i} \
        --max-missing 0.5 --min-alleles 2 --max-alleles 2 --maf 0.05 \
        --recode --out ${i}
done

ERROR: This cross-compiler package contains no program /storage/group/ibb3/default/.conda/envs/ipyrad/bin/x86_64-conda_cos6-linux-gnu-cc
ERROR: activate-gcc_linux-64.sh failed, see above for details



VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf apal_snpchip_remapped_kate_sorted.vcf
	--maf 0.05
	--max-alleles 2
	--min-alleles 2
	--max-missing 0.5
	--out apal_snpchip_remapped_kate_sorted.vcf
	--recode

After filtering, kept 107 out of 107 Individuals
Outputting VCF file...
After filtering, kept 4368 out of a possible 23218 Sites
Run Time = 2.00 seconds


In [49]:
katelinkage = pd.read_csv("/storage/group/ibb3/default/darpa/linkage_scaff/apal_kate_link_samples.csv", 
            sep = "\s", names = ["LG", "probe", "genetic position"])

/tmp/ipykernel_2024491/3639172915.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  katelinkage = pd.read_csv("/storage/group/ibb3/default/darpa/linkage_scaff/apal_kate_link_samples.csv",


In [50]:
probes = pd.read_csv("/storage/group/ibb3/default/darpa/linkage_scaff/unique_mappings_apal_probes.tab", comment = "@", sep = "\t", names = ["probe", "contig", "start", "mapq", "cigar", "seq", "mismatch"])

In [51]:
probes["new_snp_pos"] = np.nan
for i in range(len(probes)):
    probes["new_snp_pos"][i] = probes["start"][i] + probes["seq"][i].find("N")
probes = probes[~probes["cigar"].str.contains("D")][~probes["cigar"].str.contains("I")].reset_index(drop=True)
probes = probes[~probes["cigar"].str.contains("H")].reset_index(drop=True)
probes["new_snp_pos"] = probes["new_snp_pos"].astype(int)
probes = probes[["contig", "new_snp_pos", "probe"]]
probes.columns = ["Scaffold ID", "scaffold position", "probe"]
katelinkage = katelinkage.merge(probes, how = "left", on = "probe")
katelinkage.columns = ["LinkageGroup", "probe", "GeneticPosition", "ScaffoldID", "ScaffoldPosition"]
katelinkage = katelinkage[["ScaffoldID", "ScaffoldPosition", "LinkageGroup", "GeneticPosition"]]
katelinkage = katelinkage[katelinkage["ScaffoldPosition"].notna()].reset_index(drop=True)
katelinkage = katelinkage[katelinkage["ScaffoldID"] != "*"].reset_index(drop=True)
katelinkage["ScaffoldPosition"] = katelinkage["ScaffoldPosition"].astype(int)
(katelinkage.set_index("ScaffoldID")
 .to_csv("/storage/group/ibb3/default/darpa/linkage_scaff/linkage_map_1.csv"))

/tmp/ipykernel_2024491/2418261223.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  probes["new_snp_pos"][i] = probes["start"][i] + probes["seq"][i].find("N")
/tmp/ipykernel_2024491/2418261223.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  probes = probes[~probes["cigar"].str.contains("D")][~probes["cigar"].str.contains("I")].reset_index(drop=True)


In [52]:
%%bash
source ~/.bashrc
conda activate allmaps
cd /storage/group/ibb3/default/darpa/linkage_scaff/
python -m jcvi.assembly.allmaps merge linkage_map_1.csv -o linkage_merged.bed

[22:45:09] INFO     Set text.usetex=False. Font styles may be inconsistent.                                     base.py:402
           DEBUG    A total of 1199 markers written to `linkage_merged.bed`.                                allmaps.py:1281
           DEBUG    Weights file written to `weights.txt`.                                                  allmaps.py:1337


In [53]:
%%bash
source ~/.bashrc
conda activate allmaps
cd /storage/group/ibb3/default/darpa/linkage_scaff/
python -m jcvi.assembly.allmaps path --notex --renumber linkage_merged.bed \
    /storage/group/ibb3/default/genome_annotation/Apalm_dovetail_assemblies/Apalm_assembly_v2.0_180910.softmasked.fa

[22:45:12] INFO     Set text.usetex=False. Font styles may be inconsistent.                                     base.py:402
           INFO     Set text.usetex=False. Font styles may be inconsistent.                                     base.py:402
           DEBUG    Load file `linkage_merged.bed`                                                               base.py:33
           DEBUG    Map contains 1199 markers in 14 linkage groups.                                          allmaps.py:546
[10:45:12 PM] DEBUG    Retained 1,162 of 1,199 (96.9%) clean markers.                                        allmaps.py:593
              DEBUG    Load file `weights.txt`                                                                   base.py:33
              DEBUG    Imported 1 records from `weights.txt`.                                                   base.py:102
              DEBUG    Map weights: odict_items([('linkage_map_1', 1)])                                      allmaps.py:659
        

Current iteration 20: max_score=(1.0,)
Current iteration 40: max_score=(1.0,)
Current iteration 60: max_score=(1.0,)
Current iteration 80: max_score=(1.0,)
Current iteration 100: max_score=(1.0,)
Current iteration 120: max_score=(1.0,)
Current iteration 140: max_score=(1.0,)
Current iteration 160: max_score=(1.0,)
Current iteration 180: max_score=(1.0,)
Current iteration 200: max_score=(1.0,)
Current iteration 220: max_score=(1.0,)
Current iteration 240: max_score=(1.0,)
Current iteration 260: max_score=(1.0,)
Current iteration 280: max_score=(1.0,)
Current iteration 300: max_score=(1.0,)
Current iteration 320: max_score=(1.0,)
Current iteration 340: max_score=(1.0,)
Current iteration 360: max_score=(1.0,)
Current iteration 380: max_score=(1.0,)
Current iteration 400: max_score=(1.0,)
Current iteration 420: max_score=(1.0,)
Current iteration 440: max_score=(1.0,)
Current iteration 460: max_score=(1.0,)
Current iteration 480: max_score=(1.0,)
Current iteration 500: max_score=(1.0,)


[10:45:19 PM] DEBUG    Fixed orientations for 0 scaffolds.                                                   allmaps.py:469
              DEBUG    Current best fitness: (1.0,)                                                          allmaps.py:215
              DEBUG    Start EC round 1                                                                      allmaps.py:195
              DEBUG    GA setup: ngen=500 npop=100 cpus=16 seed=666                                               ec.py:177


Current iteration 20: max_score=(1.0,)
Current iteration 40: max_score=(1.0,)
Current iteration 60: max_score=(1.0,)
Current iteration 80: max_score=(1.0,)
Current iteration 100: max_score=(1.0,)
Current iteration 120: max_score=(1.0,)
Current iteration 140: max_score=(1.0,)
Current iteration 160: max_score=(1.0,)
Current iteration 180: max_score=(1.0,)
Current iteration 200: max_score=(1.0,)
Current iteration 220: max_score=(1.0,)
Current iteration 240: max_score=(1.0,)
Current iteration 260: max_score=(1.0,)
Current iteration 280: max_score=(1.0,)
Current iteration 300: max_score=(1.0,)
Current iteration 320: max_score=(1.0,)
Current iteration 340: max_score=(1.0,)
Current iteration 360: max_score=(1.0,)
Current iteration 380: max_score=(1.0,)
Current iteration 400: max_score=(1.0,)
Current iteration 420: max_score=(1.0,)
Current iteration 440: max_score=(1.0,)
Current iteration 460: max_score=(1.0,)
Current iteration 480: max_score=(1.0,)
Current iteration 500: max_score=(1.0,)


[10:45:20 PM] DEBUG    No fitness improvement: (1.0,). Exit EC.                                              allmaps.py:206
              DEBUG    Dropped 0 minor scaffolds                                                             allmaps.py:432
>chr1 (linkage_map_1-1) FINAL
Sc0a5M3_343_HRSCAF_597+
              DEBUG    Working on linkage_map_1-2 ...                                                       allmaps.py:1581
              DEBUG    Graph size: |V|=3, |E|=2                                                              allmaps.py:285
              DEBUG    TSP rescale: max_x=42, min_x=-10000, inf=20000, factor=100                                 tsp.py:48
[10:45:25 PM] INFO     Objective: 4004800                                                                        tsp.py:113
              INFO     Route distance: 1000600                                                                   tsp.py:121
              DEBUG    Start EC round 0                                       

...

[10:47:08 PM] DEBUG    No fitness improvement: (2.0,). Exit EC.                                              allmaps.py:206
              DEBUG    Dropped 1 minor scaffolds                                                             allmaps.py:432
>chr14 (linkage_map_1-14) FINAL
Sc0a5M3_398_HRSCAF_736-
              DEBUG    chr5: 36750254 => chr1                                                               allmaps.py:1612
              DEBUG    chr9: 27236598 => chr2                                                               allmaps.py:1612
              DEBUG    chr7: 23845084 => chr3                                                               allmaps.py:1612
              DEBUG    chr12: 15610647 => chr4                                                              allmaps.py:1612
              DEBUG    chr4: 14027495 => chr5                                                               allmaps.py:1612
              DEBUG    chr8: 13085460 => chr6                               

              DEBUG    cat linkage_merged.chr.agp linkage_merged.unplaced.agp >linkage_merged.agp              base.py:1185
              DEBUG    cat linkage_merged.chr.fasta linkage_merged.unplaced.fasta >linkage_merged.fasta        base.py:1185
[10:47:12 PM] DEBUG    Load file `linkage_merged.agp`                                                            base.py:33
              DEBUG    Load file                                                                                 base.py:33
                       `/storage/group/ibb3/default/genome_annotation/Apalm_dovetail_assemblies/Apalm_assembly_v           
                       2.0_180910.softmasked.fa.sizes`                                                                     
              DEBUG    Load file `/storage/group/ibb3/default/darpa/linkage_scaff/linkage_merged.fasta`          base.py:33
[10:47:15 PM] DEBUG    Load file `linkage_merged.fasta.sizes`                                                    base.py:33
        

    return _get_text_metrics_with_cache_impl(
  File "/storage/group/ibb3/default/.conda/envs/allmaps/lib/python3.9/site-packages/matplotlib/text.py", line 105, in _get_text_metrics_with_cache_impl
    return renderer_ref().get_text_width_height_descent(text, fontprop, ismath)
  File "/storage/group/ibb3/default/.conda/envs/allmaps/lib/python3.9/site-packages/matplotlib/backends/_backend_pdf_ps.py", line 107, in get_text_width_height_descent
    return super().get_text_width_height_descent(s, prop, ismath)
  File "/storage/group/ibb3/default/.conda/envs/allmaps/lib/python3.9/site-packages/matplotlib/backend_bases.py", line 645, in get_text_width_height_descent
    return self.get_texmanager().get_text_width_height_descent(
  File "/storage/group/ibb3/default/.conda/envs/allmaps/lib/python3.9/site-packages/matplotlib/texmanager.py", line 368, in get_text_width_height_descent
    dvifile = cls.make_dvi(tex, fontsize)
  File "/storage/group/ibb3/default/.conda/envs/allmaps/lib/python3.9/s

  File "/storage/group/ibb3/default/.conda/envs/allmaps/lib/python3.9/site-packages/matplotlib/texmanager.py", line 368, in get_text_width_height_descent
    dvifile = cls.make_dvi(tex, fontsize)
  File "/storage/group/ibb3/default/.conda/envs/allmaps/lib/python3.9/site-packages/matplotlib/texmanager.py", line 300, in make_dvi
    cls._run_checked_subprocess(
  File "/storage/group/ibb3/default/.conda/envs/allmaps/lib/python3.9/site-packages/matplotlib/texmanager.py", line 259, in _run_checked_subprocess
    raise RuntimeError(
RuntimeError: Failed to process string with tex because latex could not be found


CalledProcessError: Command 'b'source ~/.bashrc\nconda activate allmaps\ncd /storage/group/ibb3/default/darpa/linkage_scaff/\npython -m jcvi.assembly.allmaps path --notex --renumber linkage_merged.bed \\\n    /storage/group/ibb3/default/genome_annotation/Apalm_dovetail_assemblies/Apalm_assembly_v2.0_180910.softmasked.fa\n'' returned non-zero exit status 1.

In [55]:
!cat /storage/group/ibb3/default/darpa/linkage_scaff/linkage_merged.summary.txt

*** Summary for each individual map ***
                         o          linkage_map_1
-------------------------------------------------
            Linkage Groups                     14
          Markers (unique)                  1,199
            Markers per Mb                    4.4
             N50 Scaffolds                     16
                 Scaffolds                     67
   Scaffolds with 1 marker                     14
  Scaffolds with 2 markers                      8
  Scaffolds with 3 markers                      6
Scaffolds with >=4 markers                     39
               Total bases    270,349,958 (88.9%)
-------------------------------------------------
*** Summary for consensus map ***
                         o               Anchored               Oriented               Unplaced
-----------------------------------------------------------------------------------------------
          Markers (unique)                    809                    734            

In [54]:
%%bash
source ~/.bashrc
conda activate ipyrad
stats.sh /storage/group/ibb3/default/darpa/linkage_scaff/linkage_merged.fasta

ERROR: This cross-compiler package contains no program /storage/group/ibb3/default/.conda/envs/ipyrad/bin/x86_64-conda_cos6-linux-gnu-cc
ERROR: activate-gcc_linux-64.sh failed, see above for details
A	C	G	T	N	IUPAC	Other	GC	GC_stdev
0.3051	0.1950	0.1950	0.3049	0.0006	0.0000	0.0000	0.3900	0.0203

Main genome scaffold total:         	419
Main genome contig total:           	2366
Main genome scaffold sequence total:	304.254 MB
Main genome contig sequence total:  	304.060 MB  	0.064% gap
Main genome scaffold N/L50:         	8/12.984 MB
Main genome contig N/L50:           	400/230.835 KB
Main genome scaffold N/L90:         	42/542.891 KB
Main genome contig N/L90:           	1366/62.198 KB
Max scaffold length:                	36.75 MB
Max contig length:                  	1.429 MB
Number of scaffolds > 50 KB:        	232
% main genome in scaffolds > 50 KB: 	98.47%


Minimum 	Number        	Number        	Total         	Total         	Scaffold
Scaffold	of            	of            	Scaffold   